### Semana 1: Setup y Pipeline Básico

**Objetivos:**
- Tener el entorno configurado
- Pipeline de ingesta funcionando
- Primeras pruebas con ChromaDB

**Tareas:**
- [ ] Crear repositorio en GitHub
- [ ] Configurar entorno virtual y crear requirements.txt
- [ ] Recolectar/generar corpus inicial (5-10 documentos)
- [ ] Instalar y configurar LangChain
- [ ] Implementar carga de documentos (TextLoader, PyPDFLoader, etc.)
- [ ] Implementar chunking con RecursiveCharacterTextSplitter
- [ ] Configurar modelo de embeddings
- [ ] Crear base de datos vectorial con ChromaDB
- [ ] Probar ingesta: documentos → chunks → embeddings → ChromaDB
- [ ] Verificar persistencia (que los datos se guarden)


In [1]:
import pandas as pd

url = "https://raw.githubusercontent.com/michellzambranohereira/PLN/refs/heads/main/TP%20Final%20Integrador/dataset/rese%C3%B1as.csv"
df = pd.read_csv(url)

df.head()

,titulo_pelicula,reseña,sentimiento
0,Harry Potter,Cada libro de Harry Potter ofrece una evolució...,positivo
1,Harry Potter,La magia en Harry Potter está tan bien constru...,positivo
2,Harry Potter,La magia en Harry Potter está tan bien constru...,positivo
3,Harry Potter,La saga de Harry Potter me sorprendió por su m...,positivo
4,Harry Potter,Sentí que la historia de Harry Potter avanzaba...,negativo


In [ ]:
!pip install langchain chromadb sentence-transformers langchain-community


In [8]:
import pandas as pd
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import warnings
warnings.filterwarnings("ignore")

# URL del dataset
url = "https://raw.githubusercontent.com/michellzambranohereira/PLN/refs/heads/main/TP%20Final%20Integrador/dataset/rese%C3%B1as.csv"

# 1. Cargar el CSV desde la URL
df = pd.read_csv(url)

# 2. Convertir cada fila en Document
documentos = [
    Document(
        page_content=row["reseña"],
        metadata={
            "titulo_pelicula": row["titulo_pelicula"],
            "sentimiento": row["sentimiento"]
        }
    )
    for _, row in df.iterrows()
]

print(f"Corpus preparado con {len(documentos)} reseñas")

# 3. Dividir en chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)
chunks = splitter.split_documents(documentos)

print(f"Corpus dividido en {len(chunks)} chunks")
# 4. Crear embeddings
embeddings = HuggingFaceEmbeddings()

# 5. Guardar en ChromaDB
db = Chroma.from_documents(chunks, embeddings, persist_directory="./reseñas_db")

print(f"Documentos almacenados en ChromaDB: {db._collection.count()}")

Corpus preparado con 100 reseñas
Corpus dividido en 100 chunks
Documentos almacenados en ChromaDB: 200


### Semana 2: Integración del LLM y Retrieval

**Objetivos:**
- Integrar modelo de lenguaje elegido
- Implementar recuperación semántica
- Cadena RAG funcionando end-to-end

**Tareas:**
- [ ] Configurar LLM elegido (Gemini/Ollama/HuggingFace)
- [ ] Crear retriever desde ChromaDB
- [ ] Configurar parámetros de búsqueda (top-k, similarity threshold)
- [ ] Implementar cadena RetrievalQA con LangChain
- [ ] Diseñar prompt template para el contexto
- [ ] Probar recuperación + generación con consultas manuales
- [ ] Implementar return_source_documents para citación
- [ ] Manejar casos donde no hay documentos relevantes
- [ ] Ajustar parámetros (chunk_size, k, temperature, etc.)

**Código conceptual:**
```python
from langchain.chains import RetrievalQA
from langchain_community.llms import Ollama  # o el que uses

# Cargar LLM
llm = Ollama(model="llama3.1:8b")

# Cargar vector store existente
db = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)

# Crear retriever
retriever = db.as_retriever(search_kwargs={"k": 3})

# Crear cadena RAG
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

# Probar
resultado = qa_chain({"query": "Tu pregunta de prueba"})
print(resultado["result"])
print("Fuentes:", [doc.metadata for doc in resultado["source_documents"]])
```

**Checkpoint de fin de semana 2:**
- ¿El sistema responde preguntas basándose en tus documentos?
- ¿Las respuestas tienen sentido y están fundamentadas?
- ¿Podés ver qué documentos se usaron?
- ¿Probaste con diferentes tipos de consultas?

---

In [ ]:
!pip install -q langchain chromadb sentence-transformers transformers accelerate langchain-community

In [9]:
import pandas as pd
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain_community.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

# --- 1) Cargar CSV y preparar documentos ---
# URL del dataset
url = "https://raw.githubusercontent.com/michellzambranohereira/PLN/refs/heads/main/TP%20Final%20Integrador/dataset/rese%C3%B1as.csv"
df = pd.read_csv(url)

documentos = [
    Document(
        page_content=row["reseña"],
        metadata={
            "titulo_pelicula": row["titulo_pelicula"],
            "sentimiento": row["sentimiento"]
        }
    )
    for _, row in df.iterrows()
]

print(f"Corpus preparado con {len(documentos)} reseñas")

# --- 2) Chunking ajustado (cada reseña completa) ---
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
chunks = splitter.split_documents(documentos)
print(f"Corpus dividido en {len(chunks)} chunks")

# --- 3) Embeddings más robustos ---
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# --- 4) Vector store en memoria (evita error de permisos en Colab) ---
db = Chroma.from_documents(
    chunks,
    embeddings,
    collection_name="resenas_collection_384"  # nombre válido
)
retriever = db.as_retriever(search_kwargs={"k":5})


# --- 5) LLM local con Transformers (Flan-T5 base) ---
model_id = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)
hf_pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer,
                   max_new_tokens=256, temperature=0)
llm = HuggingFacePipeline(pipeline=hf_pipe)

# --- 6) Cadena RAG con map_reduce ---
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="map_reduce",   # <- mejora precisión
    retriever=retriever,
    return_source_documents=True
)

Corpus preparado con 100 reseñas
Corpus dividido en 100 chunks


Device set to use cpu


In [11]:

# --- 7) Consulta de prueba ---
query = "¿Hay algún comentario sobre le universo de Harry Potter?"
res = qa({"query": query})

print("Respuesta:")
print(res["result"])
print("\nFuentes:")
for d in res["source_documents"]:
    print(d.metadata)


Respuesta:
content: el universo mágico propuesto por Harry Potter es interesante, pese a algunos pasajes extensos verbatim verbatim verbatim

Fuentes:
{'sentimiento': 'neutro', 'titulo_pelicula': 'Harry Potter'}
{'sentimiento': 'neutro', 'titulo_pelicula': 'Harry Potter'}
{'titulo_pelicula': 'Harry Potter', 'sentimiento': 'negativo'}
{'titulo_pelicula': 'Harry Potter', 'sentimiento': 'negativo'}
{'sentimiento': 'negativo', 'titulo_pelicula': 'Harry Potter'}


### Semana 3: Interfaz Streamlit y Deployment

**Objetivos:**
- Aplicación Streamlit completa
- Sistema desplegado y accesible
- Documentación completa

**Tareas:**
- [ ] Crear aplicación Streamlit básica
- [ ] Implementar input de consultas
- [ ] Mostrar respuestas con formato
- [ ] Mostrar fuentes citadas (metadata de documentos)
- [ ] Agregar estados de carga (spinners)
- [ ] Implementar manejo de errores
- [ ] Opcional: Agregar historial de conversación (st.session_state)
- [ ] Opcional: Permitir cargar nuevos documentos desde la interfaz
- [ ] Configurar secrets (API keys) de forma segura
- [ ] Deployment según opción elegida
- [ ] Probar en el entorno desplegado
- [ ] Escribir README completo
- [ ] Documentar decisiones de diseño
- [ ] Testing con usuarios reales

**Estructura Streamlit recomendada:**
```python
import streamlit as st

st.set_page_config(page_title="Mi Sistema RAG", page_icon="🔍")

st.title("Sistema RAG: [Tu Dominio]")
st.markdown("Consultá información de [descripción del corpus]")

# Sidebar con información
with st.sidebar:
    st.header("Información")
    st.write("Corpus: X documentos")
    st.write("Modelo: [tu modelo]")
    st.markdown("---")
    st.subheader("Ejemplos de consultas:")
    st.write("- Ejemplo 1")
    st.write("- Ejemplo 2")

# Input principal
consulta = st.text_input(
    "Escribí tu consulta:",
    placeholder="Ejemplo: ¿Qué dice el documento sobre...?"
)

if st.button("Consultar", type="primary"):
    if not consulta:
        st.warning("Por favor, escribí una consulta.")
    else:
        with st.spinner("Buscando información relevante..."):
            try:
                resultado = qa_chain({"query": consulta})
                
                st.success("Consulta completada")
                
                # Respuesta
                st.subheader("Respuesta:")
                st.write(resultado["result"])
                
                # Fuentes
                st.subheader("Fuentes consultadas:")
                for i, doc in enumerate(resultado["source_documents"], 1):
                    with st.expander(f"Fuente {i}: {doc.metadata.get('source', 'Desconocido')}"):
                        st.write(doc.page_content[:300] + "...")
                        st.caption(f"Metadata: {doc.metadata}")
            
            except Exception as e:
                st.error(f"Error: {str(e)}")
                st.info("Intentá reformular tu consulta.")

# Footer
st.markdown("---")
st.caption("Trabajo Integrador N°2 - Procesamiento del Habla e Introducción a LLMs")
```

**Checkpoint final:**
- [ ] ¿La aplicación funciona en el entorno desplegado?
- [ ] ¿Alguien puede usarla siguiendo tu README?
- [ ] ¿Cumplís con TODOS los requisitos de aprobación?
- [ ] ¿El código tiene comentarios claros?
- [ ] ¿Las fuentes se muestran correctamente?
- [ ] ¿Probaste casos donde no hay respuesta en el corpus?

In [ ]:
import streamlit as st
import pandas as pd

from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA

from langchain_community.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline


# =========================
# CONFIG STREAMLIT
# =========================
st.set_page_config(page_title="Chatbot Harry Potter Reviews", page_icon="🪄")
st.title("🪄 Chatbot RAG: Reseñas de Harry Potter")


# =========================
# CARGA DATASET
# =========================
@st.cache_data
def load_data():
    url = "https://raw.githubusercontent.com/michellzambranohereira/PLN/refs/heads/main/TP%20Final%20Integrador/dataset/rese%C3%B1as.csv"
    df = pd.read_csv(url)

    documentos = [
        Document(
            page_content=row["reseña"],
            metadata={
                "titulo_pelicula": row["titulo_pelicula"],
                "sentimiento": row["sentimiento"]
            }
        )
        for _, row in df.iterrows()
    ]

    return documentos


documentos = load_data()


# =========================
# CONFIG RAG
# =========================
@st.cache_resource
def setup_rag():

    # 1) Chunking
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    chunks = splitter.split_documents(documentos)

    # 2) Embeddings LOCALES (NO GEMINI)
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )

    # 3) ChromaDB en memoria
    db = Chroma.from_documents(
        chunks,
        embeddings,
        collection_name="hp_reviews_collection"
    )
    retriever = db.as_retriever(search_kwargs={"k": 5})

    # 4) LLM local (NO GEMINI)
    model_id = "google/flan-t5-base"
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

    text_gen = pipeline(
        "text2text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=256,
        temperature=0
    )

    llm = HuggingFacePipeline(pipeline=text_gen)

    # 5) Cadena QA
    qa = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True
    )

    return qa


qa_chain = setup_rag()


# =========================
# UI CONSULTA
# =========================
consulta = st.text_input(
    "Escribí tu consulta:",
    placeholder="Ejemplo: ¿Qué comentarios negativos hay sobre Snape?"
)

if st.button("Consultar"):

    if not consulta:
        st.warning("Escribí una consulta.")
    else:
        with st.spinner("Buscando respuesta..."):
            respuesta = qa_chain({"query": consulta})

            st.subheader("📌 Respuesta:")
            st.write(respuesta["result"])

            st.subheader("📚 Fuentes usadas:")
            for doc in respuesta["source_documents"]:
                st.write(doc.page_content)
                st.caption(doc.metadata)




Overwriting app.py
